# Query Log Analysis

## Functionalities
- Plot number of database queries per second.
- Plot instantaneous latency of database queries.
- Plot latency distribution of database queries.

## Input
Log files are read from a directory in `../data`. This directory is assumed to have the following structure:
```
logs/
  [node-1]/
    *_service*.tar.gz
  ...
  [node-n]/
    *_service*.tar.gz
```
These tarballs contain database query log files named `queries.log`.

## Notebook Configuration

In [ ]:
########## GENERAL
# Name of the directory in `../data`
EXPERIMENT_DIRNAME = "BuzzBlogBenchmark_2021-11-03-23-37-35"

########## LATENCY
# Max expected value
MAX_LATENCY_IN_S = 1
# Bin size
LATENCY_BIN_IN_MS = 1

## Notebook Setup

In [ ]:
# Import libraries.
%matplotlib inline
import datetime
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import tarfile
import warnings
warnings.filterwarnings("ignore")

# Constants
COMPONENT_TARBALL_PATTERNS = [
    r"^.+_service.*\.tar\.gz$",
]
QUERY_LOG_PATTERN = r"^\[(.+)\] pid=(.+) tid=(.+) request_id=(.+) dbname=(.+) latency=(.+) query=\"(.+)\"$"

## Log Parsing

In [ ]:
# Parse logs
query = {"node_name": [], "timestamp": [], "request_id": [], "dbname": [], "type": [], "latency": []}
node_names = os.listdir(os.path.join(os.pardir, "data", EXPERIMENT_DIRNAME, "logs"))
for node_name in node_names:
  for tarball_name in os.listdir(os.path.join(os.pardir, "data", EXPERIMENT_DIRNAME, "logs", node_name)):
    if sum([1 if re.match(tarball_pattern, tarball_name) else 0 for tarball_pattern in COMPONENT_TARBALL_PATTERNS]):
      tarball_path = os.path.join(os.pardir, "data", EXPERIMENT_DIRNAME, "logs", node_name, tarball_name)
      with tarfile.open(tarball_path, "r:gz") as tar:
        for filename in tar.getnames():
          if filename.endswith("queries.log"):
            with tar.extractfile(filename) as log_file:
              for row in log_file:
                query_log_match = re.match(QUERY_LOG_PATTERN, row.decode("utf-8"))
                if query_log_match:
                  timestamp, pid, tid, request_id, dbname, latency, query_str = query_log_match.groups()
                  query["node_name"].append(node_name)
                  query["timestamp"].append(datetime.datetime.strptime(timestamp[:-3], "%H:%M:%S.%f"))
                  query["request_id"].append(request_id)
                  query["dbname"].append(dbname)
                  query["type"].append(query_str.strip().split()[0].upper())
                  query["latency"].append(float(latency) * 1000)

In [ ]:
# Build data frame
query = pd.DataFrame.from_dict(query)

# Get values
dbnames = sorted(query["dbname"].unique())
types = sorted(query["type"].unique())
min_timestamp = query["timestamp"].values.min()

# (Re) Build columns
query["timestamp"] = query.apply(lambda q: (q["timestamp"] - min_timestamp).total_seconds(), axis=1)
query["window"] = query.apply(lambda q: int(q["timestamp"]), axis=1)

## Number of Queries per Second

In [ ]:
########## LOCAL CONFIG
# Minimum time (in seconds)
MIN_TIME = None
# Maximum time (in seconds)
MAX_TIME = None

# Plot
fig = plt.figure(figsize=(24, len(dbnames) * 12))
for (i, dbname) in enumerate(dbnames):
    df = query[(query["dbname"] == dbname)]
    if MIN_TIME:
        df = df[(df["timestamp"] >= MIN_TIME)]
    if MAX_TIME:
        df = df[(df["timestamp"] <= MAX_TIME)]
    df = df.groupby(["window", "type"])["window"].count().unstack().fillna(0)
    df = df.reindex(range(int(df.index.min()), int(df.index.max()) + 1), fill_value=0)
    ax = fig.add_subplot(len(dbnames), 1, i + 1)
    ax.grid(alpha=0.75)
    ax.set_xlim((MIN_TIME or 0, MAX_TIME or int(df.index.max())))
    ax.set_ylim((0, int(df.values.max())))
    df.plot(ax=ax, kind="line", title="Queries per second - %s" % dbname, xlabel="Time (seconds)",
        ylabel="Queries (count)", color={"SELECT": "orange", "INSERT": "blue", "DELETE": "red", "UPDATE": "green"},legend=True, grid=True)
    plt.subplots_adjust(hspace=0.25)

## Instantaneous Latency of Queries

In [ ]:
########## LOCAL CONFIG
# Minimum time (in seconds)
MIN_TIME = None
# Maximum time (in seconds)
MAX_TIME = None

# Plot
fig = plt.figure(figsize=(24, len(dbnames) * len(types) * 12))
for (i, dbname) in enumerate(dbnames):
    for (j, type) in enumerate(types):
        df = query[(query["dbname"] == dbname) & (query["type"] == type)]
        if MIN_TIME:
            df = df[(df["timestamp"] >= MIN_TIME)]
        if MAX_TIME:
            df = df[(df["timestamp"] <= MAX_TIME)]
        if df.empty:
            continue
        df.set_index("timestamp", inplace=True)
        df.sort_index(inplace=True)
        ax = fig.add_subplot(len(dbnames) * len(types), 1, i * len(types) + j + 1)
        ax.grid(alpha=0.75)
        ax.set_xlim((MIN_TIME or 0, MAX_TIME or int(df.index.max())))
        ax.set_ylim((0, df["latency"].max()))
        df["latency"].plot(ax=ax, kind="line", title="Instantaneous Latency - %s" % dbname,
            xlabel="Time (seconds)", ylabel="Latency (milliseconds)", label=type,
            color={"SELECT": "orange", "INSERT": "blue", "DELETE": "red", "UPDATE": "green"}, legend=True, grid=True)
        plt.subplots_adjust(hspace=0.25)

## Latency Distribution of Queries

In [ ]:
########## LOCAL CONFIG
# Minimum time (in seconds)
MIN_TIME = None
# Maximum time (in seconds)
MAX_TIME = None

# Plot
fig = plt.figure(figsize=(24, len(dbnames) * len(types) * 12))
for (i, dbname) in enumerate(dbnames):
    for (j, type) in enumerate(types):
        df = query[(query["dbname"] == dbname) & (query["type"] == type)]
        if MIN_TIME:
            df = df[(df["timestamp"] >= MIN_TIME)]
        if MAX_TIME:
            df = df[(df["timestamp"] <= MAX_TIME)]
        if df.empty:
            continue
        df["latency_bin"] = df.apply(lambda r: int(r["latency"] // LATENCY_BIN_IN_MS), axis=1)
        ax = fig.add_subplot(len(dbnames) * len(types), 1, i * len(types) + j + 1)
        ax.set_yscale("log")
        ax.grid(alpha=0.75)
        ax.set_xlim((0, (1000 // LATENCY_BIN_IN_MS) * MAX_LATENCY_IN_S))
        df["latency_bin"].plot(ax=ax, kind="hist", title="Latency Distribution - %s - %s" % (dbname, type),
            xlabel="Latency (milliseconds)", ylabel="Queries (count)",
            bins=range((1000 // LATENCY_BIN_IN_MS) * MAX_LATENCY_IN_S), grid=True)
        plt.subplots_adjust(hspace=0.25)

## Statistics

In [ ]:
for (i, dbname) in enumerate(dbnames):
    print(dbname)
    for (j, type) in enumerate(types):
        df = query[(query["dbname"] == dbname) & (query["type"] == type)]
        if df.empty:
            continue
        print("  %s" % type)
        print("    Number of queries")
        print("      Total:       %7d" % df.shape[0])
        print("      Avg:         %7.2f" % (df.shape[0] / (df["timestamp"].max() - df["timestamp"].min())))
        print("    Latency (ms)")
        print("    P99.9:         %7.2f" % (df["latency"].quantile(0.999)))
        print("      P99:         %7.2f" % (df["latency"].quantile(0.99)))
        print("      P95:         %7.2f" % (df["latency"].quantile(0.95)))
        print("      P50:         %7.2f" % (df["latency"].quantile(0.50)))
        print("      Avg:         %7.2f" % (df["latency"].mean()))
        print("      Std:         %7.2f" % (df["latency"].std()))